In [1]:
!pip install anvil-uplink
!pip install textblob
!pip install tweepy
!pip install wordcloud
!pip install config

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import anvil.server
anvil.server.connect('server_57AQBXHXBXQB3SV63LDEGSMA-UP4OJMY33G3H35TG')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


In [3]:
import config
from textblob import TextBlob
import tweepy
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import re 
import pandas as pd
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import csv
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
n_words= set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud,STOPWORDS
porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 
nltk.download('punkt')
nltk.download('wordnet'
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import tweepy
api_key = 'N00JfH3Y4zMrtYxCNPbqiWXfb'
api_key_secret = 'Gh1SwlftMFJjjMEpCPtaw0ai1lnWHrSOPlJEPO42JBE5FVaspN'
access_token = '1453279345292816394-pcM7tZ81qLxrw02UwhpuZVQiuzTkL3'
access__token_secret = 'n9rCrffxa4SBov2cgXaDYtFEjt715JP45pbIqPiZWdjSY'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAPcmcAEAAAAACC8mIptctcONcVXXg5Zvh%2BkT4sY%3DrCtUvDowB8H2uY4jjOl6u5gUvogRe19XwvqPAn2ChxHDngPOur'
client = tweepy.Client(bearer_token=bearer_token,
                           consumer_key=api_key,
                           consumer_secret=api_key_secret,
                           access_token=access_token,
                           access_token_secret=access__token_secret)

In [5]:
auth_handler = tweepy.OAuthHandler(consumer_key=api_key, consumer_secret=api_key_secret)
auth_handler.set_access_token(access_token, access__token_secret)

api = tweepy.API(auth_handler)

In [6]:
search_term = 'bored apes'
tweet_amount = 1000

In [7]:
def clean(text):
  # removing @ tags and links from the text
  text= ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split()) 
  # converting all letters to lower case and relacing '-' with spaces.
  text= text.lower().replace('-', ' ')
  # removing stowards and numbers
  table= str.maketrans('', '', string.punctuation+string.digits)
  text= text.translate(table)
  # tokenizing words
  tokens = word_tokenize(text)
  # stemming the words 
  stemmed = [porter.stem(word) for word in tokens]
  words = [w for w in stemmed if not w in n_words]
  # lemmetizing words
  # lemme= [lemmatizer.lemmatize(word) for word in tokens]
  # words = [w for w in lemme if not w in n_words]
  text = ' '.join(words)
  return text 

In [8]:
@anvil.server.callable
def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word  # double check for nay links
                                and not word.startswith('#')  # removing hash tags
                                and word != 'rt'  
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS, # using stopwords provided by Word cloud its optional since we already removed stopwords :)
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    # using matplotlib to display the images in notebook itself.
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show

In [17]:
@anvil.server.callable
def sentiment(search_term): 
  tweet_amount = 100
  #tweets = tweepy.Cursor(api.search_recent_tweets, q= search_term, lang = 'en', wait_on_rate_limit=True).items(tweet_amount)
  tweets = client.search_recent_tweets(query=search_term, max_results=tweet_amount)[0]
  tweetss=[]
  for tweet in tweets:
      tweet= clean(tweet.text)
      analysis = TextBlob(tweet)
      senti= analysis.sentiment.polarity
      if senti<0 :
        emotion = "NEG"
      elif senti>0:
        emotion= "POS"
      else:
        emotion= "NEU"
      tweetss.append((tweet, senti, emotion))
  df= pd.DataFrame(tweetss, columns= ['tweets', 'senti', 'emotion'])
  # droping retweets
  df= df.drop_duplicates()
  # saving to CSV file
  df.to_csv('data.csv', index= False)
  df_neu= df[ df['emotion'] == 'NEU']
  df_neu= df_neu['tweets']
  df_pos = df[ df['emotion'] == 'POS']
  df_pos = df_pos['tweets']
  df_neg = df[ df['emotion'] == 'NEG']
  df_neg = df_neg['tweets']

  ratio_pos= len(df_pos)/len(df)
  ratio_neg= len(df_neg)/len(df)
  ratio_neu= len(df_neu)/len(df)

  percent_pos = ratio_pos * 100
  percent_neg = ratio_neg * 100 
  percent_neu = ratio_neu * 100 
  return [percent_pos, percent_neg, percent_neu]

In [18]:
sentiment(search_term)

[20.967741935483872, 12.903225806451612, 66.12903225806451]

In [19]:
@anvil.server.callable
def pos(search_term): 
  pos_result = sentiment(search_term)
  return pos_result[0]

In [20]:
pos('try')

37.755102040816325

In [21]:
@anvil.server.callable
def neg(search_term): 
  neg_result = sentiment(search_term)
  return neg_result[1]

In [22]:
@anvil.server.callable
def neu(search_term): 
  neu_result = sentiment(search_term)
  return neu_result[2]